<a href="https://colab.research.google.com/github/AlcidesChewe/Tarea-Academica-1-Topics/blob/main/topicos_tp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21

In [ ]:
from ortools.sat.python import cp_model

def aircrew_assignment():
    model = cp_model.CpModel()

    aircrew = {
        'stewards': ['Tom', 'David', 'Jeremy', 'Ron', 'Joe', 'Bill', 'Fred', 'Bob', 'Mario', 'Ed'],
        'air_hostesses': ['Carol', 'Janet', 'Tracy', 'Marilyn', 'Carolyn', 'Cathy', 'Inez', 'Jean', 'Heather', 'Juliet']
    }
    flights = list(range(1, 11))
    aircrew_numbers = [4, 5, 5, 6, 7, 4, 5, 6, 6, 7]
    min_hostesses = [1, 1, 1, 2, 3, 1, 1, 1, 2, 3]
    min_stewards = [1, 1, 1, 2, 3, 1, 1, 1, 2, 3]

    language_knowledge = {
        'Francais': ['Inez', 'Bill', 'Jean', 'Juliet'],
        'Espagnol': ['Tom', 'Jeremy', 'Mario', 'Cathy', 'Juliet'],
        'Allemand': ['Bill', 'Fred', 'Joe', 'Mario', 'Marilyn', 'Inez', 'Heather']
    }

    assignments = {}
    for flight in flights:
        for aircrew_type in aircrew:
            for member in aircrew[aircrew_type]:
                assignments[(flight, member)] = model.NewBoolVar(f'flight_{flight}_{member}')

    for flight in flights:
        model.Add(sum(assignments[(flight, member)] for member in aircrew['stewards'] + aircrew['air_hostesses']) == aircrew_numbers[flight-1])

        model.Add(sum(assignments[(flight, member)] for member in aircrew['air_hostesses']) >= min_hostesses[flight-1])
        model.Add(sum(assignments[(flight, member)] for member in aircrew['stewards']) >= min_stewards[flight-1])

        for language in language_knowledge:
            members_with_language = [member for member in aircrew['stewards'] + aircrew['air_hostesses'] if member in language_knowledge[language]]
            model.Add(sum(assignments[(flight, member)] for member in members_with_language) >= 1)

    for i in range(1, len(flights) - 1):
        for member in aircrew['stewards'] + aircrew['air_hostesses']:
            model.Add(assignments[(i, member)] + assignments[(i + 1, member)] <= 1)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        for flight in flights:
            print(f"Flight {flight}:")
            for member in aircrew['stewards'] + aircrew['air_hostesses']:
                if solver.Value(assignments[(flight, member)]) == 1:
                    print(f"\t{member}")
    else:
        print("No solution found.")

aircrew_assignment()


Flight 1:
	David
	Jeremy
	Bill
	Carol
Flight 2:
	Ron
	Joe
	Fred
	Mario
	Inez
Flight 3:
	Tom
	David
	Jeremy
	Bill
	Carol
Flight 4:
	Ron
	Joe
	Fred
	Mario
	Marilyn
	Inez
Flight 5:
	Tom
	David
	Jeremy
	Bill
	Carol
	Janet
	Tracy
Flight 6:
	Ron
	Joe
	Mario
	Inez
Flight 7:
	Tom
	David
	Jeremy
	Bill
	Carol
Flight 8:
	Ron
	Joe
	Fred
	Bob
	Mario
	Inez
Flight 9:
	Tom
	David
	Jeremy
	Bill
	Carol
	Janet
Flight 10:
	Jeremy
	Bill
	Fred
	Mario
	Cathy
	Inez
	Jean
